In [1]:
# Import needed libraries 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics
from sklearn.utils import resample
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import BaggingClassifier, RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, precision_score,recall_score,f1_score
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.metrics import roc_curve, roc_auc_score 
from sklearn.metrics import confusion_matrix 
from scipy.stats import norm
from scipy.stats import multivariate_normal
from sklearn.preprocessing import MinMaxScaler
from imblearn import under_sampling, over_sampling
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score


In [2]:
# Import the data
df_original = pd.read_csv(r'/Users/ibby/Desktop/frauddata.csv')

In [3]:
# Make a copy of the dataset, and make edits on that
df = df_original.copy()

In [4]:
# Check the head to see if everything looks good
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,...,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,...,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,...,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,...,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,...,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


# Data Preprocessing

In [5]:
# Standardize the data

df_clean = df.copy()

scaler = StandardScaler()
df_clean['std_amount'] = scaler.fit_transform(df_clean['Amount'].values.reshape(-1, 1))
df_clean['std_time'] = scaler.fit_transform(df_clean['Time'].values.reshape(-1, 1))

df_clean.drop(['Amount', 'Time'], axis=1, inplace=True)

In [6]:
# Instantiate y to the target variable
# Instantiate x to be the features of the dataset
# Drop any unneeded features

X = df_clean.drop('Class', axis=1)

y = df_clean['Class']

# train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [7]:
# create a validation set. 
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train,test_size=.2, random_state=2)

# Deep Nueral Network

# Vanilla Model

# Random Forest

In [8]:
%%time

rndf = RandomForestClassifier()

rndf.fit(X_train,y_train)

CPU times: user 1min 54s, sys: 197 ms, total: 1min 54s
Wall time: 1min 54s


RandomForestClassifier()

In [9]:
y_test_preds = rndf.predict(X_test)
rf_acc = accuracy_score(y_test, y_test_preds)
rf_prec = precision_score(y_test, y_test_preds)
rf_rec = recall_score(y_test, y_test_preds)


In [10]:
print(f'The accuarcy  score is {rf_acc}')
print(f'The precision score is {rf_prec}')
print(f'The recall score is {rf_rec}')

The accuarcy  score is 0.9995786664794073
The precision score is 0.9625
The recall score is 0.7857142857142857


In [11]:
%%time
clf = RandomForestClassifier(criterion = 'entropy',
                            max_depth = 5,
                            n_estimators=20)


# Fit the model to the training data
clf.fit(X_train, y_train)

# use the model to predict test accuracy

y_hat_test = clf.predict(X_test)
rf_acc = accuracy_score(y_test, y_hat_test)
rf_prec = precision_score(y_test, y_hat_test)
rf_rec = recall_score(y_test, y_hat_test)

print(f'The accuarcy  score is {rf_acc}')
print(f'The precision score is {rf_prec}')
print(f'The recall score is {rf_rec}')


The accuarcy  score is 0.9994557775359011
The precision score is 0.9135802469135802
The recall score is 0.7551020408163265
CPU times: user 9.33 s, sys: 25.5 ms, total: 9.35 s
Wall time: 9.35 s


In [12]:
clf = DecisionTreeClassifier(random_state=42, criterion='entropy')

In [13]:
%%time
clf.fit(X_train, y_train)

CPU times: user 6.69 s, sys: 6.44 ms, total: 6.7 s
Wall time: 6.7 s


DecisionTreeClassifier(criterion='entropy', random_state=42)

# Isolation Forest Classifier

In [14]:
# %%time
# #Building another model/classifier ISOLATION FOREST
# from sklearn.ensemble import IsolationForest
# ifc=IsolationForest(max_samples=len(X_train),
#  contamination=outlier_fraction,random_state=1)
# ifc.fit(X_train)
# scores_pred = ifc.decision_function(X_train)
# y_pred = ifc.predict(X_test)

In [15]:
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier # KNN algorithm
from sklearn.linear_model import LogisticRegression # Logistic regression algorithm
from sklearn.svm import SVC # SVM algorithm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV

In [16]:
# %%time
# # create an XGBoost model
# xgb = XGBClassifier()

# xgb.fit(X_train, y_train)

In [17]:
xgb_y_preds = xgb.predict(X_test)
xgb_acc = accuracy_score(y_test,xgb_y_preds)
xgb_prec = precision_score(y_test, xgb_y_preds)
xgb_rec = recall_score(y_test, xgb_y_preds)

print(f'The accuarcy  score is {xgb_acc}')
print(f'The precision score is {xgb_prec}')
print(f'The recall score is {xgb_rec}')

NameError: name 'xgb' is not defined

In [ ]:
# %%time
# n = 3

# knn = KNeighborsClassifier(n_neighbors = n)
# knn.fit(X_train, y_train)
# knn_yhat = knn.predict(X_test)

In [ ]:
knn_y_preds = knn.predict(X_test)
knn_acc = accuracy_score(y_test, knn_y_preds)
knn_prec = precision_score(y_test, knn_y_preds)
knn_rec = recall_score(y_test, knn_y_preds)

print(f'The accuarcy  score is {knn_acc}')
print(f'The precision score is {knn_prec}')
print(f'The recall score is {knn_rec}')